<h1> Polymoprhic Eptiope Prediction </h1>

This tutorial illustrates how to use Fred2 to integrate genomic information of a patient for epitope prediction

This tutorial will entails:
- Variant construction
- Polymorphic Transcript/Protein/Peptide construction
- Polymorphic epitope prediction


In [9]:
%matplotlib inline
%load_ext autoreload
%reload_ext autoreload
%autoreload 2

#is is needed as long as Fred2 is not installed properly
import sys 

sys.path.append("/Users/schubert/Dropbox/PhD/Porgramming/Fred2")
from Fred2.Core import Allele, Peptide, Protein,generate_peptides_from_protein
from Fred2.IO import read_lines, read_fasta
from Fred2.EpitopePrediction import EpitopePredictorFactory
from Fred2.Core import generate_transcripts_from_variants, generate_proteins_from_transcripts 
from Fred2.Core import generate_peptides_from_variants
from Fred2.IO import read_annovar_exonic
from Fred2.IO.MartsAdapter import MartsAdapter

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


<h2> Chapter 1: Generating polymorphic epitopes </h2>
<br/>


In [17]:
vars = read_annovar_exonic("./data/test_annovar.out")
trans = generate_transcripts_from_variants(vars, MartsAdapter())

In [18]:
list(generate_proteins_from_transcripts(trans))

[PROTEIN: MDWGTLQTILGGVNKHSTSIGKIWLTVLFIFRIMILVVAAKEVWGDEQADFVCNTLQPGCKNVCYDHYFPISHIRLWALQLIFVSTPALLVAMHVAYRRHEKKRKFIKGEIKSEFKDIEEIKTQKVRIEGSLWWTYTSSIFFRVIFEAAFMYVFYVMYDGFSMQRLVKCNAWPCPNTVDCFVSRPTEKTVFTVFMIAVSGICILLNVTELCYLLIRYCSGKSKKPV* (aa-seq)
 		  NM_004004:FRED2_0 (orig transcript)
 		 VARIANTS:,
 PROTEIN: MDWGTLQTILGV*TNTPPALERSGSPSSSFFAL*SSLWLQRRCGEMSRPTLSATPCSQAARTCATITTSPSPTSGYGPCS*SSCPRQRS*WPCTWPTGDMRRRGSSSRGR*RVNLRTSRRSKPRRSASKAPCGGPTQAASSSGSSSKPPSCTSSMSCTTASPCSGW*SATPGLVPTLWTALCPGPRRRLSSQCS*LQCLEFASC*MSLNCVIC*LDIVLGSQKSQF (aa-seq)
 		  NM_004004:FRED2_1 (orig transcript)
 		 VARIANTS:
 		 pos 11: Variant(g.20763686C>-),
 PROTEIN: MGEEGGSASHDEEERASVLLGHSPGCEMCSQEAFQAQRSQLVELLVSGSLEGFESVLDWLLSWEVLSWEDYEGFHLLGQPLSHLARRLLDTVWNKGTWACQKLIAAAQEAQADSQSPKLHGCWDPHSLHPARDLQSHRPAIVRRLHSHVENMLDLAWERGFVSQYECDEIRLPIFTPSQRARRLLDLATVKANGLAAFLLQHVQELPVPLALPLEAATCKKYMAKLRTTVSAQSRFLSTYDGAETLCLEDIYTENVLEVWADVGMAGPPQKSPATLGLEELFSTPGHLNDDADTVLVVGEAGSGKSTLLQRLHLLWAAGQDFQEFLFVFPFSCRQLQCMAKPLSVRTLLF

In [13]:
peptide = generate_peptides_from_protein(proteins, 9)

KeyboardInterrupt: 

In [ ]:
peptides = generate_peptides_from_variants(vars, 9, MartsAdapter())